# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [4]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-history3-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Cultural Movements explorative search") 

Consider the following exploratory information need:

> investigate the cultural movements, in particular the Renaissance, Baroque and Romanticsm. We'd like to get an overview of the main artists belonging to these movements, musicians, poets, sculptors and painters.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q37562`| Donatello    | node |
| `wd:Q19675`| Louvre Museum | node |





Also consider

```
wd:Q37562 ?p ?obj .
```

is the BGP to retrieve all **properties of Donatello**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Cultural Movement

2. Identify the BGP for Renassaince, Baroque and Romanticism

3. Identify the BGP for composer, painter, sculptor and poet

4. Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

5. Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

6. For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

7. Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

In [5]:
## startup the evaluation
# setup the file and create the empty json
ipname = "h3.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 1_6


## Task 1

In [6]:
# look at the properties of Donatello
queryString = """
SELECT  DISTINCT ?prop ?name 
WHERE {
    wd:Q37562 ?prop ?obj .
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('prop', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('prop', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('prop', 'http://www.wikidata.org/prop/direct/P1138'), ('name', 'Kunstindeks Danmark Artist ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('prop', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1248'), ('name', 'KulturNav-ID')]
[('prop', 'http://www

In [7]:
# use P135 movement
queryString = """
SELECT  DISTINCT ?obj ?name 
WHERE {
    wd:Q37562 wdt:P135 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
1


In [8]:
# look at the properties of these movement
queryString = """
SELECT  DISTINCT ?prop ?name 
WHERE {
    wd:Q1472236 ?prop ?obj .
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P5604'), ('name', 'archINFORM keyword ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('prop', 'http://www.wikidata.org/prop/direct/P576'), ('name', 'dissolved, abolished or demolished date')]
[('prop', 'http://www.wikidat

In [9]:
# use part of P361
queryString = """
SELECT  DISTINCT ?obj ?name 
WHERE {
    wd:Q1472236 wdt:P361 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
1


In [10]:
# look at the instance of 
queryString = """
SELECT  DISTINCT ?inst ?name 
WHERE {
    wd:Q1472236 wdt:P361 ?obj .
    ?obj wdt:P31 ?inst.
    ?inst sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')]
[('inst', 'http://www.wikidata.org/entity/Q32880'), ('name', 'architectural style')]
[('inst', 'http://www.wikidata.org/entity/Q968159'), ('name', 'art movement')]
3


In [11]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q2198855"
og_name = "cultural movement"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 1_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_6.json
JSON object updated


`Q2198855`-> cultural movement

## Task 2

In [12]:
# list instances of cultural movements
queryString = """
SELECT  DISTINCT ?obj ?name 
WHERE {
    ?obj wdt:P31 wd:Q2198855.
    ?obj sc:name ?name.
    FILTER(REGEX(?name,"[B,b]aroque") || REGEX(?name,"[R,r]enaiss") || REGEX(?name,"[R,r]omant") ) .
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
[('obj', 'http://www.wikidata.org/entity/Q2614483'), ('name', 'Pre-romanticism')]
[('obj', 'http://www.wikidata.org/entity/Q1404472'), ('name', 'Italian Renaissance')]
[('obj', 'http://www.wikidata.org/entity/Q430167'), ('name', 'Spanish Renaissance')]
[('obj', 'http://www.wikidata.org/entity/Q15991289'), ('name', 'Old School Renaissance')]
[('obj', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
[('obj', 'http://www.wikidata.org/entity/Q2349908'), ('name', 'French Renaissance')]
[('obj', 'http://www.wikidata.org/entity/Q979160'), ('name', 'Proto-Renaissance')]
[('obj', 'http://www.wikidata.org/entity/Q1400430'), ('name', 'Macedonian Renaissance')]
[('obj', 'http://www.wikidata.org/entity/Q846933'), ('name', 'renaissance humanism')]
[('obj', 'http://www.wikidata.org/entity/Q2046861'), ('name', 'Palaeologan Renaissance')]
[('obj', 'http://www.wikidata.org/entity/Q1431758'), ('name', 'Sa

Find everything:
- `Q37068` -> Romanticsm
- `Q37853` -> Baroque
- `Q4692` -> Renaissance

In [13]:
# list instances of cultural movements
queryString = """
SELECT ?mov ?name 
WHERE {
    VALUES ?mov {wd:Q37068 wd:Q37853 wd:Q4692}.
    ?mov sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('mov', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
[('mov', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
3


In [14]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 1_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_6.json
JSON object updated


## Task 3

In [15]:
# starts with Baroque and check how people is connected with it
queryString = """
SELECT  DISTINCT ?prop ?name 
WHERE {
    ?people wdt:P31 wd:Q5.
    ?people ?prop wd:Q37853.
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('prop', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
[('prop', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('prop', 'http://www.wikidata.org/prop/direct/P149'), ('name', 'architectural style')]
[('prop', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
5


In [16]:
# let see properties of people. Use Donatello to speed up the search
queryString = """
SELECT  DISTINCT ?prop ?name 
WHERE {
    wd:Q37562 ?prop ?x.
    FILTER(!isLiteral(?x)). 
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('prop', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('prop', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
[('prop', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('prop', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('prop', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('prop', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p

In [17]:
# starts with Baroque and check how people is connected with
queryString = """
SELECT  DISTINCT ?x ?name 
WHERE {
    ?people wdt:P31 wd:Q5.
    ?people wdt:P135 wd:Q37853.
    ?people wdt:P106 ?x.
    ?x sc:name ?name.
    FILTER(REGEX(?name,"[S,s]culpt") || REGEX(?name,"[P,p]oet") || REGEX(?name,"[C,c]ompos") || REGEX(?name,"[P,p]aint")  ) 
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('x', 'http://www.wikidata.org/entity/Q288728'), ('name', 'painter and varnisher')]
[('x', 'http://www.wikidata.org/entity/Q984276'), ('name', 'court painter')]
[('x', 'http://www.wikidata.org/entity/Q24056480'), ('name', 'porcelain painter')]
[('x', 'http://www.wikidata.org/entity/Q28947943'), ('name', 'miniature painter')]
[('x', 'http://www.wikidata.org/entity/Q22669155'), ('name', 'fresco painter')]
[('x', 'http://www.wikidata.org/entity/Q21600439'), ('name', 'landscape painter')]
[('x', 'http://www.wikidata.org/entity/Q1183704'), ('name', 'decorative painter')]
[('x', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('x', 'http://www.wikidata.org/entity/Q21680663'), ('name', 'classical composer')]
[('x', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('x', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
12


Find everything:
- `Q36834` -> composer
- `Q49757` -> poet
- `Q1281618` -> sculptor
- `Q1028181` -> painter

In [18]:
# show oonly the results
queryString = """
SELECT  ?jobs ?jName 
WHERE {
    VALUES ?jobs {wd:Q36834 wd:Q49757 wd:Q1281618 wd:Q1028181} .
    ?jobs sc:name ?jName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('jobs', 'http://www.wikidata.org/entity/Q1028181'), ('jName', 'painter')]
[('jobs', 'http://www.wikidata.org/entity/Q1281618'), ('jName', 'sculptor')]
[('jobs', 'http://www.wikidata.org/entity/Q36834'), ('jName', 'composer')]
[('jobs', 'http://www.wikidata.org/entity/Q49757'), ('jName', 'poet')]
4


In [19]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 1_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_6.json
JSON object updated


## Task 4

In [20]:
# group the results
queryString = """
SELECT  ?jobs ?jName ?movements ?mName COUNT(DISTINCT ?people) AS ?peop
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    VALUES ?jobs {wd:Q36834 wd:Q49757 wd:Q1281618 wd:Q1028181} .
    ?people wdt:P135 ?movements.
    ?people wdt:P106 ?jobs.
    ?jobs sc:name ?jName.
    ?movements sc:name ?mName.
}
GROUP BY ?jobs ?jName ?movements ?mName
"""

print("Results")
x=run_query(queryString)

Results
[('jobs', 'http://www.wikidata.org/entity/Q36834'), ('jName', 'composer'), ('movements', 'http://www.wikidata.org/entity/Q4692'), ('mName', 'Renaissance'), ('peop', '8')]
[('jobs', 'http://www.wikidata.org/entity/Q36834'), ('jName', 'composer'), ('movements', 'http://www.wikidata.org/entity/Q37853'), ('mName', 'Baroque'), ('peop', '6')]
[('jobs', 'http://www.wikidata.org/entity/Q1281618'), ('jName', 'sculptor'), ('movements', 'http://www.wikidata.org/entity/Q37853'), ('mName', 'Baroque'), ('peop', '328')]
[('jobs', 'http://www.wikidata.org/entity/Q1028181'), ('jName', 'painter'), ('movements', 'http://www.wikidata.org/entity/Q4692'), ('mName', 'Renaissance'), ('peop', '56')]
[('jobs', 'http://www.wikidata.org/entity/Q49757'), ('jName', 'poet'), ('movements', 'http://www.wikidata.org/entity/Q4692'), ('mName', 'Renaissance'), ('peop', '15')]
[('jobs', 'http://www.wikidata.org/entity/Q49757'), ('jName', 'poet'), ('movements', 'http://www.wikidata.org/entity/Q37068'), ('mName', 'Ro

In [21]:
## more elements mixed between URI and literals
objs=[]
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    m_uri = i[2][1]
    m_name = i[3][1]
    people = i[4][1]
    # add the year element
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"] = people
    objs.append(obj)
    # add the director element
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"] = m_uri
    obj["name"] = m_name
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_REFERRED ,["uri","name","value"], objs, elements_per_tuple = 2)

The index of this workflow is: 1_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_6.json
JSON object updated


## Task 5

In [22]:
# First of all, find sculptures starting from sculptors
queryString = """
SELECT DISTINCT ?prop ?name 
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    ?people wdt:P135 ?movements;
         wdt:P106 wd:Q1281618.
    ?x ?prop ?people.
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('prop', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('prop', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('prop', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('prop', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('prop', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P22'), ('name', 'father')]
[('prop', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse')]
[('prop', 'http://www.wikidata.org/prop/direct/P287'), ('name', 'designed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P301'), ('n

In [23]:
# list some sculptures 
queryString = """
SELECT DISTINCT ?x ?name 
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    ?people wdt:P135 ?movements;
         wdt:P106 wd:Q1281618.
    ?x wdt:P170 ?people.
    ?x sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q10359993'), ('name', 'Portrait of Helena Fourment')]
[('x', 'http://www.wikidata.org/entity/Q12029726'), ('name', "Kohl's Fountain")]
[('x', 'http://www.wikidata.org/entity/Q21711520'), ('name', 'The Virgin and Child with Saint Elizabeth and Saint John the Baptist')]
[('x', 'http://www.wikidata.org/entity/Q21724938'), ('name', 'Mercury Moving away (Mercurius Abituriens)')]
[('x', 'http://www.wikidata.org/entity/Q21730885'), ('name', 'Landscape with Huts')]
[('x', 'http://www.wikidata.org/entity/Q21741378'), ('name', 'Landscape with Cymon and Iphigenia')]
[('x', 'http://www.wikidata.org/entity/Q17320340'), ('name', 'Cimon and Pero (Roman Charity)')]
[('x', 'http://www.wikidata.org/entity/Q26791267'), ('name', 'The Descent from the Cross by Peter Paul Rubens (Saint-Omer)')]
[('x', 'http://www.wikidata.org/entity/Q29866374'), ('name', 'Crowning of the hero')]
[('x', 'http://www.wikidata.org/entity/Q29866384'), ('name', 'The apocalyptic woman

In [24]:
# find the class of sculpture
queryString = """
SELECT DISTINCT ?inst ?name 
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    ?people wdt:P135 ?movements;
         wdt:P106 wd:Q1281618.
    ?x wdt:P170 ?people.
    ?x wdt:P31 ?inst.
    ?inst sc:name ?name.
    FILTER(REGEX(?name,"[S,s]culp")).
}
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q2293362'), ('name', 'sculptural group')]
[('inst', 'http://www.wikidata.org/entity/Q860861'), ('name', 'sculpture')]
[('inst', 'http://www.wikidata.org/entity/Q245117'), ('name', 'relief sculpture')]
[('inst', 'http://www.wikidata.org/entity/Q837249'), ('name', 'marble sculpture')]
[('inst', 'http://www.wikidata.org/entity/Q27031439'), ('name', 'group of sculptures')]
[('inst', 'http://www.wikidata.org/entity/Q928357'), ('name', 'bronze sculpture')]
[('inst', 'http://www.wikidata.org/entity/Q19479037'), ('name', 'sculpture series')]
[('inst', 'http://www.wikidata.org/entity/Q16675884'), ('name', 'religious sculpture')]
8


In [25]:
# list some sculpture using the subclasses
queryString = """
SELECT DISTINCT ?x ?name 
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    ?people wdt:P135 ?movements;
         wdt:P106 wd:Q1281618.
    ?x wdt:P170 ?people.
    ?x wdt:P31/wdt:P279* wd:Q860861.
    ?x sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q12029726'), ('name', "Kohl's Fountain")]
[('x', 'http://www.wikidata.org/entity/Q29912556'), ('name', 'prophet or Saint')]
[('x', 'http://www.wikidata.org/entity/Q473237'), ('name', 'Statue of Bartolomeo Colleoni')]
[('x', 'http://www.wikidata.org/entity/Q3647660'), ('name', 'Bust of a Young Woman')]
[('x', 'http://www.wikidata.org/entity/Q501204'), ('name', 'Putto with a Dolphin')]
[('x', 'http://www.wikidata.org/entity/Q65923107'), ('name', 'Bust of a Young noble lady')]
[('x', 'http://www.wikidata.org/entity/Q3701200'), ('name', 'Lady with Primroses')]
[('x', 'http://www.wikidata.org/entity/Q63854807'), ('name', 'Flayed Horse I')]
[('x', 'http://www.wikidata.org/entity/Q71556144'), ('name', 'Saint John the Baptist')]
[('x', 'http://www.wikidata.org/entity/Q2415504'), ('name', 'David with the head of Goliath')]
10


In [26]:
# find the location
queryString = """
SELECT DISTINCT ?prop ?name 
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    ?people wdt:P135 ?movements;
         wdt:P106 wd:Q1281618.
    ?x wdt:P170 ?people.
    ?x wdt:P31/wdt:P279* wd:Q860861.
    ?x ?prop ?y.
    ?prop sc:name ?name.
    FILTER(REGEX(?name,"locate")).
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P669'), ('name', 'located on street')]
2


In [27]:
# identify what is in the located in the administrative territory
queryString = """
SELECT DISTINCT ?y ?name 
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    ?people wdt:P135 ?movements;
         wdt:P106 wd:Q1281618.
    ?x wdt:P170 ?people.
    ?x wdt:P31/wdt:P279* wd:Q860861.
    ?x wdt:P131 ?y.
    ?y sc:name ?name.
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q744598'), ('name', 'Hradčany')]
[('y', 'http://www.wikidata.org/entity/Q973974'), ('name', 'Prague 1')]
[('y', 'http://www.wikidata.org/entity/Q2444636'), ('name', 'Prague 2')]
[('y', 'http://www.wikidata.org/entity/Q616877'), ('name', 'Louny')]
[('y', 'http://www.wikidata.org/entity/Q753219'), ('name', 'New Town')]
[('y', 'http://www.wikidata.org/entity/Q748211'), ('name', 'Old Town')]
[('y', 'http://www.wikidata.org/entity/Q1741'), ('name', 'Vienna')]
[('y', 'http://www.wikidata.org/entity/Q220'), ('name', 'Rome')]
[('y', 'http://www.wikidata.org/entity/Q621'), ('name', 'Versailles')]
[('y', 'http://www.wikidata.org/entity/Q3434113'), ('name', 'Ljubljana City Municipality')]
[('y', 'http://www.wikidata.org/entity/Q3413129'), ('name', "Saint-Thomas-d'Aquin")]
[('y', 'http://www.wikidata.org/entity/Q11911'), ('name', 'Canton of Bern')]
[('y', 'http://www.wikidata.org/entity/Q666217'), ('name', 'Bern-Mittelland administrative district')]
[

In [28]:
# identify what is in the located in the administrative territory
queryString = """
SELECT DISTINCT ?c ?name 
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    ?people wdt:P135 ?movements;
         wdt:P106 wd:Q1281618.
    ?x wdt:P170 ?people.
    ?x wdt:P31/wdt:P279* wd:Q860861.
    ?x wdt:P131 ?y.
    ?y wdt:P17 ?c.
    ?c sc:name ?name.
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('c', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('c', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('c', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland')]
[('c', 'http://www.wikidata.org/entity/Q419'), ('name', 'Peru')]
[('c', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('c', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('c', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium')]
[('c', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('c', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
13


In [29]:
# sculptures made in Italy
queryString = """
SELECT DISTINCT ?movements ?name  COUNT(DISTINCT ?x) AS ?sculptures
WHERE {
    VALUES ?movements {wd:Q37068 wd:Q37853 wd:Q4692} .
    ?people wdt:P135 ?movements;
         wdt:P106 wd:Q1281618.
    OPTIONAL{
        ?x wdt:P170 ?people;
            wdt:P31/wdt:P279* wd:Q860861;
            wdt:P131 ?city.
        ?city wdt:P17 wd:Q38.
    }
    ?movements sc:name ?name.
}
GROUP BY ?movements ?name
"""

print("Results")
x=run_query(queryString)

Results
[('movements', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism'), ('sculptures', '0')]
[('movements', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance'), ('sculptures', '5')]
[('movements', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sculptures', '10')]
3


In [30]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    sculptures = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= sculptures
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 1_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_6.json
JSON object updated


## Task 6

In [31]:
# find poet of romanticism
queryString = """
SELECT DISTINCT ?people ?name ?country ?cName
WHERE {
    ?people wdt:P135 wd:Q37068;
         wdt:P106 wd:Q49757;
        wdt:P27 ?country.
    ?country sc:name ?cName.
    ?people sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('people', 'http://www.wikidata.org/entity/Q44403'), ('name', 'Heinrich Heine'), ('country', 'http://www.wikidata.org/entity/Q12548'), ('cName', 'Holy Roman Empire')]
[('people', 'http://www.wikidata.org/entity/Q1705018'), ('name', 'Amrita Pritam'), ('country', 'http://www.wikidata.org/entity/Q129286'), ('cName', 'British India')]
[('people', 'http://www.wikidata.org/entity/Q535'), ('name', 'Victor Hugo'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France')]
[('people', 'http://www.wikidata.org/entity/Q1063392'), ('name', 'Charles Augustin Sainte-Beuve'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France')]
[('people', 'http://www.wikidata.org/entity/Q18197574'), ('name', 'Ferdinand de Gramont'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France')]
[('people', 'http://www.wikidata.org/entity/Q49767'), ('name', 'François-René de Chateaubriand'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France')]
[

In [32]:
# find property continent
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
    ?people wdt:P135 wd:Q37068;
         wdt:P106 wd:Q49757;
        wdt:P27 ?country.
    ?country ?prop ?c.
    ?prop sc:name ?name.
    FILTER(REGEX(?name,"[C,c]ontin")). 
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
1


In [33]:
# find Europe
queryString = """
SELECT DISTINCT ?c ?name
WHERE {
    ?people wdt:P135 wd:Q37068;
         wdt:P106 wd:Q49757;
        wdt:P27 ?country.
    ?country wdt:P30 ?c.
    ?c sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('c', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
[('c', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('c', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('c', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('c', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('c', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('c', 'http://www.wikidata.org/entity/Q828'), ('name', 'Americas')]
8


In [36]:
# group per Europe countries
queryString = """
SELECT DISTINCT ?country ?name COUNT(DISTINCT ?people) AS ?poets
WHERE {
    ?people wdt:P135 wd:Q37068;
         wdt:P106 wd:Q49757;
        wdt:P27 ?country.
    ?country wdt:P30 wd:Q46.
    ?country sc:name ?name.
}
GROUP BY ?country ?name
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q174193'), ('name', 'United Kingdom of Great Britain and Ireland'), ('poets', '7')]
[('country', 'http://www.wikidata.org/entity/Q12560'), ('name', 'Ottoman Empire'), ('poets', '1')]
[('country', 'http://www.wikidata.org/entity/Q221457'), ('name', 'Congress Poland'), ('poets', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('poets', '16')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal'), ('poets', '4')]
[('country', 'http://www.wikidata.org/entity/Q28513'), ('name', 'Austria-Hungary'), ('poets', '3')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania'), ('poets', '3')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('poets', '4')]
[('country', 'http://www.wikidata.org/entity/Q131964'), ('name', 'Austrian Empire'), ('poets', '6')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('name', 'Soviet Union'), ('poets', '4')]
[('cou

In [37]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    poets = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= poets
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 1_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_6.json
JSON object updated


## Task 7

In [40]:
#I found the property location P276, now I modify the previous query to get all the sculptures that are in the louvre museum. 
queryString = """
SELECT ?movement ?movementsName (COUNT(DISTINCT ?sculptures) AS ?scul)
WHERE {

   # bind something
   
   OPTIONAL{
       ?sculptures wdt:P31/wdt:P279* wd:Q860861 .
       ?sculptures wdt:P276 wd:Q19675. 

       ?sculptures wdt:P135 ?movement .
   }
   #renaissance  #Baroque   #Romanticism
   VALUES ?movement { wd:Q4692    wd:Q37853   wd:Q37068} 
   
   # get the label
   ?movement sc:name ?movementsName .
   
}
GROUP BY ?movement ?movementsName
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'http://www.wikidata.org/entity/Q37068'), ('movementsName', 'Romanticism'), ('scul', '0')]
[('movement', 'http://www.wikidata.org/entity/Q4692'), ('movementsName', 'Renaissance'), ('scul', '0')]
[('movement', 'http://www.wikidata.org/entity/Q37853'), ('movementsName', 'Baroque'), ('scul', '0')]
3


In [41]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    scul = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= scul
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 1_6
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_6.json
JSON object updated
